In [1]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd

# 1️⃣ Baca file Excel sumber
file_path = 'report_to_job_mapping.xlsx'  # ubah sesuai nama file sumber kamu

# Sheet 1: relasi antar-job
df = pd.read_excel(file_path, sheet_name='Job_relationship')    

# Sheet 2: daftar target job
target_jobs = pd.read_excel(file_path, sheet_name='Turunan_job1')['job_name'].dropna().tolist()

# Pastikan nama kolom lowercase
df.columns = df.columns.str.lower()
if 'job_name' not in df.columns or 'job_controller' not in df.columns:
    raise ValueError("Sheet 'relationship' harus memiliki kolom 'job_name' dan 'job_controller'")

# 2️⃣ Fungsi untuk membangun path hierarchy
def build_path(df, job_name):
    path = [job_name]
    visited = set([job_name])

    while True:
        # cari parent berdasarkan job terakhir di path
        parent = df.loc[df['job_name'] == path[-1], 'job_controller']

        # jika tidak ada parent → stop
        if parent.empty:
            break

        parent_job = str(parent.values[0]).strip()

        # jika parent kosong, NaN, atau 'not available' → stop
        if (
            pd.isna(parent_job)
            or parent_job == ''
            or parent_job.lower() == 'not available'
        ):
            break

        # jika sudah dikunjungi → deteksi loop
        if parent_job in visited:
            path.append(f"(loop to {parent_job})")
            break

        # tambahkan ke path dan lanjut
        path.append(parent_job)
        visited.add(parent_job)

    return path

# 3️⃣ Bangun hierarchy untuk semua target job
all_paths = []
for job in target_jobs:
    path = build_path(df, job)
    row = {'target_job': job}
    for i, name in enumerate(path):
        row[f'Level_{i+1}'] = name
    # ambil root job (paling atas)
    row['Root_Job'] = path[-1]
    # hitung jumlah level
    row['Level_Count'] = len(path)
    all_paths.append(row)

# 4️⃣ Gabungkan hasil
result = pd.DataFrame(all_paths)

# 5️⃣ Simpan ke file Excel baru
output_file = 'job_hierarchy_result_mcrs_v1.xlsx'
result.to_excel(output_file, sheet_name='job_hierarchy_result', index=False)

print("✅ Hierarchy berhasil dibuat!")
print(f"💾 Hasil disimpan di file baru: {output_file}")


✅ Hierarchy berhasil dibuat!
💾 Hasil disimpan di file baru: job_hierarchy_result_mcrs_v1.xlsx


In [5]:
import pandas as pd

# 1️⃣ Baca file Excel
file_path = 'report_to_job_mapping.xlsx'  # ganti sesuai nama file kamu

# Sheet report-job mapping
df_report = pd.read_excel(file_path, sheet_name='report_match_job')

# Sheet daftar job aktif
df_active = pd.read_excel(file_path, sheet_name='Job_run_time_2025')

# 2️⃣ Normalisasi kolom
df_active.columns = df_active.columns.str.lower()
active_jobs = df_active['job_name'].dropna().astype(str).str.strip().unique().tolist()

# 3️⃣ Gabungkan semua kolom matched_jobs jadi satu list
job_cols = [col for col in df_report.columns if col.lower().startswith('matched_jobs')]
df_report['all_jobs'] = df_report[job_cols].astype(str).apply(
    lambda x: [j.strip() for j in x if j != 'nan' and j.strip() != ''],
    axis=1
)

# 4️⃣ Buat dataframe terpisah per kombinasi report–job
rows = []
for _, row in df_report.iterrows():
    report = row['report_name']
    for job in row['all_jobs']:
        status = "Active" if job in active_jobs else "Inactive"
        rows.append({'report_name': report, 'job_name': job, 'status': status})

# 5️⃣ Gabungkan hasil
result = pd.DataFrame(rows)

# 6️⃣ Simpan ke Excel baru
output_file = 'report_job_status_lookup.xlsx'
result.to_excel(output_file, index=False)

print("✅ Hasil berhasil dibuat!")
print(f"💾 File disimpan sebagai: {output_file}")


✅ Hasil berhasil dibuat!
💾 File disimpan sebagai: report_job_status_lookup.xlsx


In [11]:
pip install pdf2docx


   ---------------------------------------- 0/3 [python-docx]
   ---------------------------------------- 0/3 [python-docx]
   ---------------------------------------- 0/3 [python-docx]
   ---------------------------------------- 0/3 [python-docx]
   ---------------------------------------- 0/3 [python-docx]
   ------------- -------------------------- 1/3 [fire]
   ------------- -------------------------- 1/3 [fire]
   ------------- -------------------------- 1/3 [fire]
   -------------------------- ------------- 2/3 [pdf2docx]
   -------------------------- ------------- 2/3 [pdf2docx]
   -------------------------- ------------- 2/3 [pdf2docx]
   ---------------------------------------- 3/3 [pdf2docx]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pdf2image import convert_from_path
import pytesseract
from docx import Document

pdf_file = 'Memo 1.pdf'
docx_file = 'output.docx'

# Konversi setiap halaman PDF jadi gambar
pages = convert_from_path(pdf_file)

# Buat dokumen Word baru
doc = Document()

for i, page in enumerate(pages):
    text = pytesseract.image_to_string(page, lang='eng')  # bisa ganti ke 'ind' kalau Bahasa Indonesia
    doc.add_paragraph(text)
    print(f"✅ Halaman {i+1} selesai diproses")

# Simpan hasilnya
doc.save(docx_file)
print("🎉 Konversi OCR selesai! Hasil tersimpan di", docx_file)


✅ Halaman 1 selesai diproses
✅ Halaman 2 selesai diproses
✅ Halaman 3 selesai diproses
✅ Halaman 4 selesai diproses
✅ Halaman 5 selesai diproses
✅ Halaman 6 selesai diproses
✅ Halaman 7 selesai diproses
🎉 Konversi OCR selesai! Hasil tersimpan di output.docx
